# Tile Generation Tutorial

Welcome to the tile generation tutorial!

As a whole slide image is too large for deep learning model training, a slide is often divded into a set of small tiles, and used for training. For tile-based whole slide image analysis, generating tiles and labels is an important and laborious step. With LUNA tiling CLIs and tutorials, you can easily generate tile labels and get your data ready for downstream analysis. In this notebook, we will see how to generate tiles and labels using LUNA tiling CLIs. Here are the main steps we will review:

1. Load slides
2. Generate tiles, labels
3. Collect tiles for model training

Through out this notebook, we will use different method parameter files. Please refer to the example parameter files in the `configs` directory to follow these steps.


In [2]:
env DATASET_URL=file:///home/aukermaa/vmount/PRO_12-123/

env: DATASET_URL=file:///home/aukermaa/vmount/PRO_12-123/


This is the main tiling step. The CLI generates tiles, populates otsu and purple scores along with the regional annotation label. An otsu score is calculated using the otsu foreground/background detection algorithm commonly used to filter out the background of the slide. Purple scores are calculated to provide additional guidance to H&E slide analysis.


In [18]:
!generate_tiles \
file:~/vmount/PRO_12-123/data/toy_data_set/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs \
--tile_size 128 --requested_magnification 20 \
-o /home/aukermaa/vmount/PRO_12-123/tiling/test/tiles


2022-04-21 18:21:34,036 - INFO - root - Initalized logger, log file at: data-processing.log
2022-04-21 18:21:35,815 - INFO - luna.common.utils - Started CLI Runner wtih <function generate_tiles at 0x7f8321273310>
2022-04-21 18:21:35,817 - INFO - luna.common.utils - Validating params...
2022-04-21 18:21:35,819 - INFO - luna.common.utils -  -> Set input_slide_image (<class 'str'>) = file:~/vmount/PRO_12-123/data/toy_data_set/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs
2022-04-21 18:21:35,820 - INFO - luna.common.utils -  -> Set output_dir (<class 'str'>) = /home/aukermaa/vmount/PRO_12-123/tiling/test/tiles
2022-04-21 18:21:35,821 - INFO - luna.common.utils -  -> Set tile_size (<class 'int'>) = 128
2022-04-21 18:21:35,822 - INFO - luna.common.utils -  -> Set requested_magnification (<class 'float'>) = 20.0
2022-04-21 18:21:35,823 - INFO - luna.common.utils - Expanding inputs...
2022-04-21 18:21:35,824 - INFO - luna.common.utils - Attempting to read metadata at file:~/vmount/PRO_12-123/data

In [7]:
!detect_tissue \
~/vmount/PRO_12-123/data/toy_data_set/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs \
/home/aukermaa/vmount/PRO_12-123/tiling/test/tiles \
--requested_magnification 2 \
--filter_query "otsu_score > 0.1" \
-o /home/aukermaa/vmount/PRO_12-123/tiling/test/detect

2022-04-21 16:45:25,051 - INFO - root - Initalized logger, log file at: data-processing.log
2022-04-21 16:45:26,761 - INFO - luna.common.utils - Started CLI Runner wtih <function detect_tissue at 0x7efc7b348160>
2022-04-21 16:45:26,763 - INFO - luna.common.utils - Validating params...
2022-04-21 16:45:26,764 - INFO - luna.common.utils -  -> Set input_slide_image (<class 'str'>) = /home/aukermaa/vmount/PRO_12-123/data/toy_data_set/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs
2022-04-21 16:45:26,765 - INFO - luna.common.utils -  -> Set input_slide_tiles (<class 'str'>) = /home/aukermaa/vmount/PRO_12-123/tiling/test/tiles
2022-04-21 16:45:26,767 - INFO - luna.common.utils -  -> Set requested_magnification (<class 'float'>) = 2.0
2022-04-21 16:45:26,768 - INFO - luna.common.utils -  -> Set filter_query (<class 'str'>) = otsu_score > 0.1
2022-04-21 16:45:26,769 - INFO - luna.common.utils -  -> Set output_dir (<class 'str'>) = /home/aukermaa/vmount/PRO_12-123/tiling/test/detect
2022-04-21 16:4

In [8]:
!label_tiles \
../PRO_12-123/data/toy_data_set/table/ANNOTATIONS /home/aukermaa/vmount/PRO_12-123/tiling/test/detect \
-o /home/aukermaa/vmount/PRO_12-123/tiling/test/label

2022-04-21 16:46:42,161 - INFO - root - Initalized logger, log file at: data-processing.log
2022-04-21 16:46:42,162 - INFO - luna.common.utils - Started CLI Runner wtih <function generate_tile_labels at 0x7f91b750faf0>
2022-04-21 16:46:42,163 - INFO - luna.common.utils - Validating params...
2022-04-21 16:46:42,165 - INFO - luna.common.utils -  -> Set input_slide_annotation_dataset (<class 'str'>) = ../PRO_12-123/data/toy_data_set/table/ANNOTATIONS
2022-04-21 16:46:42,167 - INFO - luna.common.utils -  -> Set input_slide_tiles (<class 'str'>) = /home/aukermaa/vmount/PRO_12-123/tiling/test/detect
2022-04-21 16:46:42,168 - INFO - luna.common.utils -  -> Set output_dir (<class 'str'>) = /home/aukermaa/vmount/PRO_12-123/tiling/test/label
2022-04-21 16:46:42,171 - INFO - luna.common.utils - Expanding inputs...
2022-04-21 16:46:42,173 - INFO - luna.common.utils - Attempting to read metadata at ../PRO_12-123/data/toy_data_set/table/ANNOTATIONS/metadata.yml
2022-04-21 16:46:42,179 - INFO - luna

In [3]:
!save_tiles \
/home/aukermaa/vmount/PRO_12-123/data/toy_data_set/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs \
/home/aukermaa/vmount/PRO_12-123/tiling/test/label \
--num_cores 16 --batch_size 200 --dataset_id PRO_TILES \
-o /home/aukermaa/vmount/PRO_12-123/tiling/test/saved_tiles

2022-04-21 18:13:58,465 - INFO - root - Initalized logger, log file at: data-processing.log
2022-04-21 18:14:00,292 - INFO - luna.common.utils - Started CLI Runner wtih <function save_tiles at 0x7f277d08b280>
2022-04-21 18:14:00,294 - INFO - luna.common.utils - Validating params...
2022-04-21 18:14:00,296 - INFO - luna.common.utils -  -> Set input_slide_image (<class 'str'>) = /home/aukermaa/vmount/PRO_12-123/data/toy_data_set/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs
2022-04-21 18:14:00,297 - INFO - luna.common.utils -  -> Set input_slide_tiles (<class 'str'>) = /home/aukermaa/vmount/PRO_12-123/tiling/test/label
2022-04-21 18:14:00,298 - INFO - luna.common.utils -  -> Set output_dir (<class 'str'>) = /home/aukermaa/vmount/PRO_12-123/tiling/test/saved_tiles
2022-04-21 18:14:00,300 - INFO - luna.common.utils -  -> Set num_cores (<class 'int'>) = 16
2022-04-21 18:14:00,301 - INFO - luna.common.utils -  -> Set batch_size (<class 'int'>) = 200
2022-04-21 18:14:00,302 - INFO - luna.common.

In [4]:
from luna.common.utils import LunaCliClient

def pipeline (slide_id, input_slide, input_annotations):
    client = LunaCliClient("~/vmount/PRO_12-123/tiling", slide_id)

    client.run(
        "bootstrap::slide", 
        input_slide, 
    )

    client.run(
        "bootstrap::annotations", 
        input_annotations, 
    )


    client.run(
        "generate_tiles::source_tiles", "slide", 
        tile_size=128, 
        requested_magnification=20
    )
    
    client.run(
        "detect_tissue::detected_tiles", "slide", "source_tiles",
        filter_query="otsu_score > 0.1", 
        requested_magnification=2
    )

    client.run(
        "label_tiles::labled_tiles", "annotations", "detected_tiles",
    )

    client.run(
        "save_tiles::saved_tiles", "slide", "labled_tiles",
        num_cores=16, batch_size=200, dataset_id='PRO_TILES'
    )

In [19]:
from concurrent.futures import ThreadPoolExecutor

df_slides = pd.read_parquet("../PRO_12-123/data/toy_data_set/table/SLIDES/slide_ingest_PRO_12-123.parquet")
        
with ThreadPoolExecutor(5) as pool:
    
    for index, row in df_slides.iterrows():
        
        pool.submit(pipeline, index, row.slide_image, "../PRO_12-123/data/toy_data_set/table/ANNOTATIONS")



generate_tiles file:/home/aukermaa/vmount/PRO_12-123/data/toy_data_set/01OV008-7579323e-2fae-43a9-b00f-a15c28.svs --tile_size 128 --requested_magnification 20 -o /home/aukermaa/vmount/PRO_12-123/tiling/01OV008-7579323e-2fae-43a9-b00f-a15c28/source_tiles
generate_tiles file:/home/aukermaa/vmount/PRO_12-123/data/toy_data_set/01OV002-ed65cf94-8bc6-492b-9149-adc16f.svs --tile_size 128 --requested_magnification 20 -o /home/aukermaa/vmount/PRO_12-123/tiling/01OV002-ed65cf94-8bc6-492b-9149-adc16f/source_tiles
generate_tiles file:/home/aukermaa/vmount/PRO_12-123/data/toy_data_set/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs --tile_size 128 --requested_magnification 20 -o /home/aukermaa/vmount/PRO_12-123/tiling/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77/source_tiles
generate_tiles file:/home/aukermaa/vmount/PRO_12-123/data/toy_data_set/01OV007-9b90eb78-2f50-4aeb-b010-d642f9.svs --tile_size 128 --requested_magnification 20 -o /home/aukermaa/vmount/PRO_12-123/tiling/01OV007-9b90eb78-2f50-4aeb-b010-d642

2022-04-21 18:31:12,094 - INFO - root - Initalized logger, log file at: data-processing.log
2022-04-21 18:31:12,103 - INFO - root - Initalized logger, log file at: data-processing.log
2022-04-21 18:31:12,111 - INFO - root - Initalized logger, log file at: data-processing.log
2022-04-21 18:31:12,139 - INFO - root - Initalized logger, log file at: data-processing.log
2022-04-21 18:31:12,158 - INFO - root - Initalized logger, log file at: data-processing.log
2022-04-21 18:31:14,149 - INFO - luna.common.utils - Started CLI Runner wtih <function generate_tiles at 0x7fa9ff2c4310>
2022-04-21 18:31:14,149 - INFO - luna.common.utils - Started CLI Runner wtih <function generate_tiles at 0x7fb816b24310>
2022-04-21 18:31:14,151 - INFO - luna.common.utils - Started CLI Runner wtih <function generate_tiles at 0x7f5929276310>
2022-04-21 18:31:14,151 - INFO - luna.common.utils - Validating params...
2022-04-21 18:31:14,151 - INFO - luna.common.utils - Validating params...
2022-04-21 18:31:14,151 - INF

2022-04-21 18:31:14,359 - INFO - luna.common.utils - Code block 'transform::generate_tiles' took: 0.1741073541343212s
2022-04-21 18:31:14,359 - INFO - luna.common.utils - ------------------------------------------------------------
2022-04-21 18:31:14,359 - INFO - luna.common.utils -  Done with transform, running post-transform functions... 
2022-04-21 18:31:14,359 - INFO - luna.common.utils - ------------------------------------------------------------
2022-04-21 18:31:14,363 - INFO - luna.common.utils - Code block 'transform::generate_tiles' took: 0.19807152077555656s
2022-04-21 18:31:14,364 - INFO - luna.common.utils - Done.
2022-04-21 18:31:14,365 - INFO - luna.common.utils - ------------------------------------------------------------
2022-04-21 18:31:14,365 - INFO - luna.common.utils -  Done with transform, running post-transform functions... 
2022-04-21 18:31:14,365 - INFO - luna.common.utils - ------------------------------------------------------------
2022-04-21 18:31:14,369 

detect_tissue file:/home/aukermaa/vmount/PRO_12-123/data/toy_data_set/01OV008-308ad404-7079-4ff8-8232-12ee2e.svs /home/aukermaa/vmount/PRO_12-123/tiling/01OV008-308ad404-7079-4ff8-8232-12ee2e/source_tiles --filter_query otsu_score > 0.1 --requested_magnification 2 -o /home/aukermaa/vmount/PRO_12-123/tiling/01OV008-308ad404-7079-4ff8-8232-12ee2e/detected_tiles
detect_tissue file:/home/aukermaa/vmount/PRO_12-123/data/toy_data_set/01OV002-ed65cf94-8bc6-492b-9149-adc16f.svs /home/aukermaa/vmount/PRO_12-123/tiling/01OV002-ed65cf94-8bc6-492b-9149-adc16f/source_tiles --filter_query otsu_score > 0.1 --requested_magnification 2 -o /home/aukermaa/vmount/PRO_12-123/tiling/01OV002-ed65cf94-8bc6-492b-9149-adc16f/detected_tiles
detect_tissue file:/home/aukermaa/vmount/PRO_12-123/data/toy_data_set/01OV008-7579323e-2fae-43a9-b00f-a15c28.svs /home/aukermaa/vmount/PRO_12-123/tiling/01OV008-7579323e-2fae-43a9-b00f-a15c28/source_tiles --filter_query otsu_score > 0.1 --requested_magnification 2 -o /home/au

2022-04-21 18:31:16,029 - INFO - root - Initalized logger, log file at: data-processing.log
2022-04-21 18:31:16,264 - INFO - root - Initalized logger, log file at: data-processing.log
2022-04-21 18:31:16,335 - INFO - root - Initalized logger, log file at: data-processing.log
2022-04-21 18:31:16,385 - INFO - root - Initalized logger, log file at: data-processing.log
2022-04-21 18:31:16,552 - INFO - root - Initalized logger, log file at: data-processing.log
2022-04-21 18:31:17,707 - INFO - luna.common.utils - Started CLI Runner wtih <function detect_tissue at 0x7f39c412b280>
2022-04-21 18:31:17,709 - INFO - luna.common.utils - Validating params...
2022-04-21 18:31:17,710 - INFO - luna.common.utils -  -> Set input_slide_image (<class 'str'>) = file:/home/aukermaa/vmount/PRO_12-123/data/toy_data_set/01OV008-308ad404-7079-4ff8-8232-12ee2e.svs
2022-04-21 18:31:17,712 - INFO - luna.common.utils -  -> Set input_slide_tiles (<class 'str'>) = /home/aukermaa/vmount/PRO_12-123/tiling/01OV008-308ad

...

100%|██████████| 20648/20648 [00:03<00:00, 5909.95it/s]
2022-04-21 18:33:38,305 - INFO - generate_tile_labels -                  x_coord  y_coord  ...  regional_label  intersection_area
2022-04-21 18:33:38,305 - INFO - generate_tile_labels - address                            ...                                   
2022-04-21 18:33:38,305 - INFO - generate_tile_labels - x24_y56_z20.0       6144    14336  ...             fat           0.001128
2022-04-21 18:33:38,305 - INFO - generate_tile_labels - x24_y57_z20.0       6144    14592  ...             fat           0.337458
2022-04-21 18:33:38,305 - INFO - generate_tile_labels - x24_y58_z20.0       6144    14848  ...             fat           0.536735
2022-04-21 18:33:38,305 - INFO - generate_tile_labels - x24_y59_z20.0       6144    15104  ...             fat           0.511210
2022-04-21 18:33:38,305 - INFO - generate_tile_labels - x24_y60_z20.0       6144    15360  ...             fat           0.161498
2022-04-21 18:33:38,305 - INFO - g

save_tiles file:/home/aukermaa/vmount/PRO_12-123/data/toy_data_set/01OV007-9b90eb78-2f50-4aeb-b010-d642f9.svs /home/aukermaa/vmount/PRO_12-123/tiling/01OV007-9b90eb78-2f50-4aeb-b010-d642f9/labled_tiles --num_cores 16 --batch_size 200 --dataset_id PRO_TILES -o /home/aukermaa/vmount/PRO_12-123/tiling/01OV007-9b90eb78-2f50-4aeb-b010-d642f9/saved_tiles


2022-04-21 18:33:39,421 - INFO - root - Initalized logger, log file at: data-processing.log
2022-04-21 18:33:41,084 - INFO - luna.common.utils - Started CLI Runner wtih <function save_tiles at 0x7f5937985310>
2022-04-21 18:33:41,086 - INFO - luna.common.utils - Validating params...
2022-04-21 18:33:41,088 - INFO - luna.common.utils -  -> Set input_slide_image (<class 'str'>) = file:/home/aukermaa/vmount/PRO_12-123/data/toy_data_set/01OV007-9b90eb78-2f50-4aeb-b010-d642f9.svs
2022-04-21 18:33:41,089 - INFO - luna.common.utils -  -> Set input_slide_tiles (<class 'str'>) = /home/aukermaa/vmount/PRO_12-123/tiling/01OV007-9b90eb78-2f50-4aeb-b010-d642f9/labled_tiles
2022-04-21 18:33:41,091 - INFO - luna.common.utils -  -> Set output_dir (<class 'str'>) = /home/aukermaa/vmount/PRO_12-123/tiling/01OV007-9b90eb78-2f50-4aeb-b010-d642f9/saved_tiles
2022-04-21 18:33:41,092 - INFO - luna.common.utils -  -> Set num_cores (<class 'int'>) = 16
2022-04-21 18:33:41,093 - INFO - luna.common.utils -  -> Se

100%|██████████| 104/104 [00:07<00:00, 14.58it/s]

2022-04-21 18:33:50,226 - INFO - generate_tiles -                 x_coord  ...                                         tile_store
2022-04-21 18:33:50,226 - INFO - generate_tiles - address                  ...                                                   
2022-04-21 18:33:50,226 - INFO - generate_tiles - x6_y152_z20.0      1536  ...  /home/aukermaa/vmount/PRO_12-123/tiling/01OV00...
2022-04-21 18:33:50,226 - INFO - generate_tiles - x6_y153_z20.0      1536  ...  /home/aukermaa/vmount/PRO_12-123/tiling/01OV00...
2022-04-21 18:33:50,226 - INFO - generate_tiles - x6_y154_z20.0      1536  ...  /home/aukermaa/vmount/PRO_12-123/tiling/01OV00...
2022-04-21 18:33:50,226 - INFO - generate_tiles - x6_y155_z20.0      1536  ...  /home/aukermaa/vmount/PRO_12-123/tiling/01OV00...
2022-04-21 18:33:50,226 - INFO - generate_tiles - x6_y156_z20.0      1536  ...  /home/aukermaa/vmount/PRO_12-123/tiling/01OV00...
2022-04-21 18:33:50,226 - INFO - generate_tiles - ...                 ...  ...            

In [22]:
import pandas as pd
df_tiles = pd.read_parquet("~/vmount/PRO_12-123/datasets/PRO_TILES/").query("intersection_area > 0")
print (df_tiles['regional_label'].value_counts())
df_tiles

tumor     2120
stroma     860
fat        751
Name: regional_label, dtype: int64


,SEGMENT_ID,dsa_collection_uuid,slide_id,address,x_coord,y_coord,xy_extent,tile_size,tile_units,otsu_score,regional_label,intersection_area,tile_store
4480,62618802005d04d493d48694-01OV002-bd8cdc70-3d46...,62618802005d04d493d48694,01OV002-bd8cdc70-3d46-40ae-99c4-90ef77,x52_y114_z20.0,13312,29184,256,128,px,0.66,stroma,0.055345,/home/aukermaa/vmount/PRO_12-123/tiling/01OV00...
4602,62618802005d04d493d48694-01OV002-bd8cdc70-3d46...,62618802005d04d493d48694,01OV002-bd8cdc70-3d46-40ae-99c4-90ef77,x53_y113_z20.0,13568,28928,256,128,px,0.81,stroma,0.212376,/home/aukermaa/vmount/PRO_12-123/tiling/01OV00...
4603,62618802005d04d493d48694-01OV002-bd8cdc70-3d46...,62618802005d04d493d48694,01OV002-bd8cdc70-3d46-40ae-99c4-90ef77,x53_y114_z20.0,13568,29184,256,128,px,0.56,stroma,0.965976,/home/aukermaa/vmount/PRO_12-123/tiling/01OV00...
4604,62618802005d04d493d48694-01OV002-bd8cdc70-3d46...,62618802005d04d493d48694,01OV002-bd8cdc70-3d46-40ae-99c4-90ef77,x53_y115_z20.0,13568,29440,256,128,px,0.45,stroma,0.344496,/home/aukermaa/vmount/PRO_12-123/tiling/01OV00...
4719,62618802005d04d493d48694-01OV002-bd8cdc70-3d46...,62618802005d04d493d48694,01OV002-bd8cdc70-3d46-40ae-99c4-90ef77,x54_y112_z20.0,13824,28672,256,128,px,0.94,stroma,0.061422,/home/aukermaa/vmount/PRO_12-123/tiling/01OV00...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
87948,62618802005d04d493d48694-01OV008-7579323e-2fae...,62618802005d04d493d48694,01OV008-7579323e-2fae-43a9-b00f-a15c28,x111_y79_z20.0,28416,20224,256,128,px,0.59,stroma,0.622304,/home/aukermaa/vmount/PRO_12-123/tiling/01OV00...
87949,62618802005d04d493d48694-01OV008-7579323e-2fae...,62618802005d04d493d48694,01OV008-7579323e-2fae-43a9-b00f-a15c28,x111_y80_z20.0,28416,20480,256,128,px,0.78,stroma,1.000000,/home/aukermaa/vmount/PRO_12-123/tiling/01OV00...
87950,62618802005d04d493d48694-01OV008-7579323e-2fae...,62618802005d04d493d48694,01OV008-7579323e-2fae-43a9-b00f-a15c28,x111_y81_z20.0,28416,20736,256,128,px,0.56,stroma,0.577473,/home/aukermaa/vmount/PRO_12-123/tiling/01OV00...
88043,62618802005d04d493d48694-01OV008-7579323e-2fae...,62618802005d04d493d48694,01OV008-7579323e-2fae-43a9-b00f-a15c28,x112_y80_z20.0,28672,20480,256,128,px,0.26,stroma,0.251255,/home/aukermaa/vmount/PRO_12-123/tiling/01OV00...


Congratulations! Now you have 2120 tumor, 860 stroma, and 751 fat tiles images and labels ready to train your model.